First let's load in our required libraries for data loading and model creation.

In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

## Load in Data

Then, we'll load the Arabic poems text file and convert it into pre-process for our network to use. 

In [ ]:
# open text file and read in data as `text`
with open('nor.txt', 'r',encoding="utf-8") as f:
    text = f.read()

Let's check out the first 100 characters, make sure everything is peachy.

In [ ]:
text[:100]

'يابير طيك طي واعماقك اعماق\nويعيش بك داب وحمامه وعصفور\nفاضت عيونك ماك وابتلت احداق\nوتنهدت منك محاله ا'

It looks like a typical Arabic poem structure, Yay!

## Tokenization

In the cells, below, I'm creating a couple **dictionaries** to convert the characters to and from integers.

In [ ]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
char2int = dict(enumerate(chars))
int2char = {ch: ii for ii, ch in char2int.items()}

# encode the text
encoded = np.array([int2char[ch] for ch in text])

Now, we can see the above part of the poem encoded in numbers

In [ ]:
encoded[:100]

array([40, 20, 36, 40, 44, 29, 43, 40, 30, 29, 43, 40, 29, 11, 20,  2,  3,
       20, 19, 30, 29, 20,  2,  3, 20, 19, 35, 11, 40,  2, 40, 22, 29, 36,
       30, 29,  7, 20, 36, 29, 11, 26,  3, 20,  3, 46, 29, 11,  2, 41, 33,
       11, 44, 35, 33, 20,  5, 28, 29,  2, 40, 11, 25, 30, 29,  3, 20, 30,
       29, 11, 20, 36, 28, 39, 28, 29, 20, 26,  7, 20, 19, 35, 11, 28, 25,
       46,  7, 28, 29,  3, 25, 30, 29,  3, 26, 20, 39, 46, 29, 20])

## Pre-processing the data

in char-RNN, the LSTM expects an input that is **one-hot encoded**
for more about one-hot-encoding, check out this post https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/

In [ ]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

## Making training mini-batches


To train on this data, we also want to create mini-batches for training. 
<img src="assets/sequence_batching@1x.png" width=500px>


<br>

In this example, we'll take the encoded characters (passed in as the `arr` parameter) and split them into multiple sequences, given by `batch_size`. Each of our sequences will be `seq_length` long.

### Creating Batches

1. The first thing we need to do is discard some of the text so we only have completely full mini-batches. 


2. After that, we need to split `arr` into $N$ batches. 


3. Now that we have this array, we can iterate through it to get our mini-batches. 


### Train and test batches
4. create two batches x,y where x is the input batch and y is the training batch which is exactly x but shifted with one character



In [ ]:
def get_batches(arr, batch_size, seq_length):

    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

### Visualise the output

let's try to get batches out of the 100 character of encoded data and see what's happening

In [ ]:
batches = get_batches(encoded, 6, 20)
x, y = next(batches)
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[40 20 36 40 44 29 43 40 30 29]
 [40 29 18 46 21 11  7 29 20 39]
 [44  5 29  3 21 25 29 13  3 21]
 [21 36 14 21 44 29  7 20 44 18]
 [17 31 39 23 29 26 40 25 20 29]
 [ 3 29  3 21 20 29 40 37 25 21]]

y
 [[20 36 40 44 29 43 40 30 29 43]
 [29 18 46 21 11  7 29 20 39 25]
 [ 5 29  3 21 25 29 13  3 21  3]
 [36 14 21 44 29  7 20 44 18 23]
 [31 39 23 29 26 40 25 20 29 11]
 [29  3 21 20 29 40 37 25 21 11]]


And as we can see that $y$ is the same as $x$ but only shifted with one character 

---
## Defining the network with PyTorch

Below is where i define the network.

<img src="assets/charRNN.png" width=500px >

Next, i will use PyTorch to define the architecture of the network. i start by defining the layers and operations we want. Then, defining a method for the forward pass.

In [ ]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [ ]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)
        
        ## put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden
        

In [ ]:
def train(net, data, epochs=5, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    valid_loss_min = np.Inf # track change in validation loss


    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = train_loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = valid_loss =  criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        model_name = 'best_loss_so_far.net'

        checkpoint = {'n_hidden': net.n_hidden,
                       'n_layers': net.n_layers,
                       'state_dict': net.state_dict(),
                       'tokens': net.chars}

        with open(model_name, 'wb') as f:
            torch.save(model_name, f)


## Training 

the next 2 cell should define the architecture and  train the model we defined above, the number of $LSTM$ layers **n_layers** is up to you, i trained it with 5 layers on google colab with 50 $epochs$ and it took **4** hours



In [ ]:
# define and print the net
n_hidden=512
n_layers=5

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(47, 512, num_layers=5, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=47, bias=True)
)


In [ ]:
batch_size = 128
seq_length = 100
n_epochs = 50 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

<div class="alert alert-block alert-info">
**Note:** the following architecure with the text file used is estimated to take 375 hours on my machine, so that's why google colab is used here, and iam gonna load my trained model and use it to predict.
</div>

## Load the sample

In [ ]:
loaded = CharRNN(net.chars,net.n_hidden,net.n_layers)
loaded.load_state_dict(net.state_dict())

<All keys matched successfully>

The following two functions is taked from **UDACITY - DeepLearning with PyTorch** to sample only the input and make it in correct foramt then take that input and generate output characters as size specified

In [ ]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [ ]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [ ]:
print(sample(loaded, 500, prime='قلبي', top_k=40))

قلبي لن هقانه  داخ ساقيه صبب يماعبت
ويل من حفرب بقي الساقا ووبقى باعب
النو سنرير فحع العبح لقوخ ننه تنرول انومي لشلاب تنمر فارابٍ طيي الشاجل تجيق
صاوخ فاكحت ابتووك ما أأانا إي
ووشهه وامعب الراسا ولر وووح
الححدهن
يا معلل القجد واسغك وماح قظغد
عيح مك افيدت من عبلين أ
جلي مهي غمل كحديا قمال
نقج بين عفيره يضل يا واسات حصع وعج السلي
الصد ما بقوات الموي تناريه معلد لفبل ولا لمفي ديدتاماج يعكد الطوره
و
اتيه تسرت مثر حل المماد من قف
قال لا زببكا وجمرده ما صلضك وشيهك وانلست
اقلي اش الغيوخ ثفر اللالن ابال
ماهد
